# Imports

In [1]:
config = {
    'dhdt': {
        'depth': 3,
        'learning_rate': 1e-2,
        
        'initializer': 'GlorotUniform', #GlorotUniform
        
        'loss': 'binary_crossentropy',#'mae',
        'optimizer': 'adam',        
        
        'beta_1': 10,
        'beta_2': 10,
        
        'squeeze_factor': 1,
        
        'batch_size': 512,
        'epochs': 1_000,
        'early_stopping_epochs': 20,
    },
    
    
    
    'make_classification': {
        'number_of_variables': 5,
        'n_samples': 5_000,
    },

    'computation': {
        'random_seed': 42,
        'num_eval': 10,
        'trials': 5,
        'n_jobs': 10,
        'verbosity': 0,
    },
}



In [2]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '' #'true'

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities_updated import *
from utilities.DHDT_updated import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy

# Evaluation

## make_classification

In [3]:
config_test = deepcopy(config)
config_test['make_classification']['n_samples'] = 1_000
config_test['dhdt']['epochs'] = 1000

dataset_dict = {}
model_dict = {}

scores_dict = {'sklearn': {},
               'DHDT': {}}

dataset_dict = get_preprocessed_dataset('make_classification',
                                        random_seed=config_test['computation']['random_seed'],
                                        config=config_test['make_classification'],
                                        verbosity=1)

model_dict['sklearn'] = DecisionTreeClassifier(max_depth=3, 
                                               random_state=config_test['computation']['random_seed'])

model_dict['sklearn'].fit(dataset_dict['X_train'], 
                          dataset_dict['y_train'])



model_dict['DHDT'] = DHDT(dataset_dict['X_train'].shape[1],

                            depth = config_test['dhdt']['depth'],

                            learning_rate = config_test['dhdt']['learning_rate'],
                            optimizer = config_test['dhdt']['optimizer'],
                          
                            initializer = config_test['dhdt']['initializer'],

                            beta_1 = config_test['dhdt']['beta_1'],
                            beta_2 = config_test['dhdt']['beta_2'],

                            squeeze_factor = config_test['dhdt']['squeeze_factor'],

                            loss = config_test['dhdt']['loss'],#'mae',

                            random_seed = config_test['computation']['random_seed'],
                            verbosity = 2)        


scores_dict['history'] = model_dict['DHDT'].fit(dataset_dict['X_train'], 
                                              dataset_dict['y_train'], 
                                              batch_size=config_test['dhdt']['batch_size'], 
                                              epochs=config_test['dhdt']['epochs'], 
                                              early_stopping_epochs=config_test['dhdt']['early_stopping_epochs'], 
                                              valid_data=(dataset_dict['X_valid'], dataset_dict['y_valid']))



scores_dict['sklearn']['accuracy_test'] = model_dict['sklearn'].score(dataset_dict['X_test'], 
                                                                 dataset_dict['y_test'])


dataset_dict['y_test_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_test'])
scores_dict['DHDT']['accuracy_test'] = accuracy_score(dataset_dict['y_test'], np.round(dataset_dict['y_test_dhdt']))


Original Data Shape (selected):  (1000, 5)
Original Data Shape (encoded):  (1000, 5)
Original Data Class Distribution:  503  (true) / 497  (false)
(700, 5) (700,)
(100, 5) (100,)
(200, 5) (200,)
True Ratio:  0.5057142857142857
self.split_values [[-0.118365705 -0.327275 -0.0287323594 -0.191525042 0.643059075]
 [0.624890864 0.162399709 -0.200222313 0.13237381 -0.402328253]
 [-0.597573519 0.112024307 -0.294433653 -0.329256743 -0.183671594]
 ...
 [0.692691505 -0.164107323 -0.640871167 0.525869906 -0.341084152]
 [0.527692497 0.207867265 -0.113402486 -0.365598619 -0.572048545]
 [0.681766689 -0.48504582 -0.283170521 -0.186742544 0.629850566]]
self.split_index_array [[-0.584261656 -0.45098424 -0.276760638 -0.190948665 -0.419533104]
 [0.0610572696 0.660722315 0.306663692 -0.358332 -0.589413524]
 [-0.399939686 -0.276240736 0.127445638 -0.553679764 -0.330857486]
 ...
 [-0.0230084658 -0.547267556 -0.350479186 0.698540151 0.199066281]
 [0.344266236 0.595816195 -0.0235255361 -0.674943 0.388583]
 [0.

epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 00 | Loss: 0.69305 |
Epoch: 01 | Loss: 0.69294 |
Epoch: 02 | Loss: 0.69295 |
Epoch: 03 | Loss: 0.69291 |
Epoch: 04 | Loss: 0.69290 |
Epoch: 05 | Loss: 0.69286 |
Epoch: 06 | Loss: 0.69272 |
Epoch: 07 | Loss: 0.69278 |
Epoch: 08 | Loss: 0.69270 |
Epoch: 09 | Loss: 0.69265 |
Epoch: 10 | Loss: 0.69259 |
Epoch: 11 | Loss: 0.69260 |
Epoch: 12 | Loss: 0.69241 |
Epoch: 13 | Loss: 0.69233 |
Epoch: 14 | Loss: 0.69232 |
Epoch: 15 | Loss: 0.69209 |
Epoch: 16 | Loss: 0.69220 |
Epoch: 17 | Loss: 0.69225 |
Epoch: 18 | Loss: 0.69199 |
Epoch: 19 | Loss: 0.69145 |
Epoch: 20 | Loss: 0.69131 |
Epoch: 21 | Loss: 0.69090 |
Epoch: 22 | Loss: 0.69110 |
Epoch: 23 | Loss: 0.69111 |
Epoch: 24 | Loss: 0.69061 |
Epoch: 25 | Loss: 0.69036 |
Epoch: 26 | Loss: 0.68937 |
Epoch: 27 | Loss: 0.68810 |
Epoch: 28 | Loss: 0.68768 |
Epoch: 29 | Loss: 0.68758 |
Epoch: 30 | Loss: 0.68589 |
Epoch: 31 | Loss: 0.68385 |
Epoch: 32 | Loss: 0.68315 |
Epoch: 33 | Loss: 0.68196 |
Epoch: 34 | Loss: 0.68019 |
Epoch: 35 | Loss: 0.

In [4]:
function_values_dhdt = tf.reshape(tf.constant([], dtype=tf.int32),[0,])
function_values_dhdt

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

In [5]:
function_values_dhdt = tf.reshape(tf.constant([], dtype=tf.int32),[0,])
function_values_dhdt = tf.concat([function_values_dhdt, [0]], axis=0)
function_values_dhdt

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>

In [6]:
tf.reshape([1,2,3], [3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [7]:
tf.reshape(tf.constant([[0],[1],[3]]), )

TypeError: Missing required positional argument

In [ ]:
tf.reshape([[0],[1],[3]], [3])

In [ ]:
tf.squeeze(tf.where(True, 0,1))

In [ ]:
tf.concat([, tf.constant([1])], axis=0)

In [ ]:
tf.ensure_shape([[0,1,1], [0,1,1]], [None,3])

In [ ]:
tf.reshape(tf.constant([1,2,3]), [3])

In [ ]:
tf.reduce_sum([[0,1,1], [0,1,1]], axis=None, keepdims=False).shape

In [ ]:
tf.ensure_shape(tf.reduce_sum([[0,1,1], [0,1,1]], axis=None, keepdims=False), [])

In [ ]:
z

In [ ]:
if True:
    parallel_eval_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_synthetic = parallel_eval_synthetic(delayed(evaluate_synthetic_parallel)(index = index,
                                                                                                random_seed_data = config['computation']['random_seed']+index,
                                                                                                random_seed_model = config['computation']['random_seed'],#+random_seed_model,
                                                                                                config = config,
                                                                                                verbosity = -1) for index in range(config['computation']['num_eval']))

    for i, synthetic_result in enumerate(evaluation_results_synthetic):
        if i == 0:
            model_dict_synthetic = synthetic_result[0]
            scores_dict_synthetic = synthetic_result[1]
            dataset_dict_synthetic = synthetic_result[2]
        else: 
            model_dict_synthetic = mergeDict(model_dict_synthetic, synthetic_result[0])
            scores_dict_synthetic = mergeDict(scores_dict_synthetic, synthetic_result[1])
            dataset_dict_synthetic = mergeDict(dataset_dict_synthetic, synthetic_result[2])


    metrics = ['accuracy_test']
    index = [i for i in range(config['computation']['num_eval'])]
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_synthetic[i]['DHDT'][metrics[0]] for i in range(config['computation']['num_eval'])]

    scores_sklearn = [scores_dict_synthetic[i]['sklearn'][metrics[0]] for i in range(config['computation']['num_eval'])]


    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['num_eval'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['num_eval'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_synthetic = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_synthetic)
    display(scores_dataframe_synthetic[scores_dataframe_synthetic.columns[1::3]])
    display(scores_dataframe_synthetic.describe())

## Real-World Eval

In [ ]:
if False:

    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    #identifier_list = ['Habermans Survival']

    parallel_eval_real_world = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_real_world = parallel_eval_real_world(delayed(evaluate_real_world_parallel)(identifier_list=identifier_list, 
                                                                                                   random_seed_model=config['computation']['random_seed']+i,
                                                                                                   config = config,
                                                                                                   verbosity = -1) for i in range(config['computation']['trials']))


    for i, real_world_result in enumerate(evaluation_results_real_world):
        if i == 0:
            model_dict_real_world = real_world_result[0]
            scores_dict_real_world = real_world_result[1]
            dataset_dict_real_world = real_world_result[2]
        else: 
            model_dict_real_world = mergeDict(model_dict_real_world, real_world_result[0])
            scores_dict_real_world = mergeDict(scores_dict_real_world, real_world_result[1])
            dataset_dict_real_world = mergeDict(dataset_dict_real_world, real_world_result[2])

    metrics = ['accuracy_test']
    index = identifier_list
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_real_world[identifier]['DHDT'][metrics[0]] for identifier in identifier_list]

    scores_sklearn = [scores_dict_real_world[identifier]['sklearn'][metrics[0]] for identifier in identifier_list]

    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_real_world = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_real_world)
    display(scores_dataframe_real_world[scores_dataframe_real_world.columns[1::3]])



In [ ]:
if False:
    identifier = identifier_list[0]#"Absenteeism"
    plt.figure(figsize=(15,8))
    image = model_dict_real_world[identifier]['DHDT'].plot(normalizer_list=dataset_dict_real_world[identifier]['normalizer_list'])
    display(image)

    plt.figure(figsize=(15,8))
    plot_tree(model_dict_real_world[identifier]['sklearn'], fontsize=10) 
    plt.show()

# Hyperparameter Optimization

In [ ]:
parameter_dict = {
        'depth': [3],
        'learning_rate': [0.05, 0.01, 0.005, 0.001, 0.0005],#[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
        
        'loss': ['binary_crossentropy'],#['binary_crossentropy', 'rmse'],#'mae',
        'optimizer': ['adam', 'sgd'],#['adam', 'sgd'],        
        
        'beta_1': [10, 50],#[10, 50, 100],
        'beta_2': [10, 50],#[10, 50, 100],
        
        'squeeze_factor': [0.2, 0.5, 1, 2, 5],#[0.2, 0.5, 1, 2, 5],    
}

parameter_grid = ParameterGrid(parameter_dict)

In [ ]:
if False:
    parallel_hpo_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_hpo_synthetic = parallel_hpo_synthetic(delayed(evaluate_parameter_setting_synthetic)(parameter_setting) for parameter_setting in parameter_grid)

    sorted_evaluation_results_hpo_synthetic = sorted(evaluation_results_hpo_synthetic, key=lambda x: x[0], reverse=True)
    print(sorted_evaluation_results_hpo_synthetic[:5])

In [ ]:
if False:
    identifier_list = [
                        'Titanic',
                        'Absenteeism',
                        #'Adult',
                      ]

    sorted_evaluation_results_hpo_real_dict = {}

    for identifier in identifier_list:
        parallel_hpo_real = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
        evaluation_results_hpo_real = parallel_hpo_real(delayed(evaluate_parameter_setting_real_world)(parameter_setting, identifier) for parameter_setting in parameter_grid)

        sorted_evaluation_results_hpo_real = sorted(evaluation_results_hpo_real, key=lambda x: x[0], reverse=True)

        sorted_evaluation_results_hpo_real_dict[identifier] = sorted_evaluation_results_hpo_real

        print(sorted_evaluation_results_hpo_real[:5])
        